In [12]:
import pandas as pd

data_weather = pd.read_csv("../Data/daily_weatherdata_2022.csv")
data_weather.head(20)

,Month,Day,LC_RAININ,LC_DAILYRAIN,LC_WINDDIR,LC_WINDSPEED,Hour,LC_TEMP_QCL3
0,1,1,2.195156e-06,0.000043,-6.263419,0.324702,11.491857,12.194145
1,1,2,6.714853e-04,0.003387,-28.223994,0.707815,11.500000,11.893159
2,1,3,7.292375e-04,0.006724,-34.624291,0.650430,11.500000,9.539620
3,1,4,4.543651e-04,0.003578,-25.189413,0.326204,11.500000,6.768886
4,1,5,1.273384e-04,0.000255,-43.943098,0.602983,11.500000,3.889968
5,1,6,2.225057e-05,0.000377,-17.698413,0.270271,11.500000,3.660429
6,1,7,1.740363e-04,0.002242,-25.585955,0.598921,11.500000,3.926373
7,1,8,6.638322e-04,0.002638,-8.783447,0.606362,11.500000,3.672283
8,1,9,1.223781e-04,0.005141,-28.891440,0.527744,11.500000,4.340409
9,1,10,7.865646e-06,0.000123,7.590774,0.126139,11.500000,0.243363


In [4]:
min_wind = data_weather["LC_WINDDIR"].min()
print(min_wind)
max_wind = data_weather["LC_WINDDIR"].max()
print(max_wind)

-43.94309807256236
38.94784580498866


In [7]:
#average wind speed in Leuven according to the KMI for 1991 to 2020
avg_wind_speed_Uccle = [4.7, 4.5, 4.2, 3.5, 3.3, 3.1, 3.1, 3.1, 3.3, 3.8, 4.1, 4.6]

print(len(avg_wind_speed_Uccle))

avg_wind = data_weather.groupby("Month")["LC_WINDSPEED"].mean()

avg_wind.head()

12


Month
1    0.336536
2    0.740747
3    0.250732
4    0.369274
5    0.240475
Name: LC_WINDSPEED, dtype: float64

In [15]:
import plotly.graph_objects as go

months = ["Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"]

# Create trace for the average wind speed over the last 20 years
trace1 = go.Scatter(
    x=months,
    y=avg_wind_speed_Uccle,
    mode='markers+lines',
    name='Average Wind Speed (Last 20 years)'
)

# Create trace for the wind speed in 2022
trace2 = go.Scatter(
    x=months,
    y=avg_wind,
    mode='markers+lines',
    name='Wind Speed (2022)'
)

# Create layout
layout = go.Layout(
    title='Wind Speed Comparison - Less wind in the city?',
    xaxis=dict(title='Month'),
    yaxis=dict(title='Wind Speed'),
    hovermode='closest',
    legend=dict(orientation='h', y=1.1, yanchor='top')
)

# Create figure and add traces
fig = go.Figure(data=[trace1, trace2], layout=layout)

# Display the figure
fig.show()

In [56]:
base_direction = 135+90 #SW and correction polar plot having 0 on the right

data_weather["WINDDIR_transformed"]=data_weather["LC_WINDDIR"]

max_wind_avg = avg_wind.max()
avg_wind_rescaled = avg_wind/max_wind_avg

print(avg_wind_rescaled.head())

import numpy as np

# Calculate weighted average wind direction for each month
monthly_avg_direction = data_weather.groupby(data_weather["Month"]) \
    .apply(lambda x: np.average(x["LC_WINDDIR"], weights=x["LC_WINDSPEED"])) \
    .reset_index(name="Weighted Avg Direction")

monthly_avg_direction["Weighted Avg Direction"] +=base_direction

print(monthly_avg_direction.head())

Month
1    0.454320
2    1.000000
3    0.338485
4    0.498517
5    0.324639
Name: LC_WINDSPEED, dtype: float64
   Month  Weighted Avg Direction
0      1              199.094201
1      2              192.905464
2      3              240.815949
3      4              218.818258
4      5              210.071449


In [57]:
print(monthly_avg_direction["Weighted Avg Direction"][0])
print(avg_wind_rescaled[1])

""" fig = go.Figure(data=go.Barpolar(
        r=[avg_wind_rescaled[1]],
        theta=[monthly_avg_direction["Weighted Avg Direction"][0]],
        marker_color='rgb(106,137,204)',
        marker_line_color='rgb(8,48,107)',
        name='Wind Rose'
    ))
fig.update_layout(
    polar=dict(radialaxis=dict(visible=True)),
    showlegend=False
)
fig.show() """

arrow_direction = monthly_avg_direction["Weighted Avg Direction"][0]
arrow_length = avg_wind_rescaled[1]

arrow_x = arrow_length * np.cos(np.deg2rad(arrow_direction))
arrow_y = arrow_length * np.sin(np.deg2rad(arrow_direction))

""" marker_symbols = ""

if -22.5 <= arrow_direction <= 22.5:
    marker_symbols = 'triangle-down'
elif 22.5 < arrow_direction <= 67.5:
    marker_symbols = 'triangle-sw'
elif 67.5 < arrow_direction <= 112.5:
    marker_symbols = 'triangle-left'
elif 112.5 < arrow_direction <= 157.5:
    marker_symbols = 'triangle-nw'
elif 157.5 < arrow_direction <= 202.5:
    marker_symbols = 'triangle-up'
elif 202.5 < arrow_direction <= 247.5:
    marker_symbols = 'triangle-ne'
elif 247.5 < arrow_direction <= 292.5:
    marker_symbols = 'triangle-right'
else:
    marker_symbols = 'triangle-se' """

fig = go.Figure()

# Add scatter plot points
fig.add_trace(go.Scatterpolar(
    r=[0, avg_wind_rescaled[1]],  # Radius
    theta=[0, arrow_direction],  # Angle
    mode='lines',
    line=dict(
        color='black',
        width=3
    ),
))

fig.add_trace(go.Scatterpolar(
    r=[arrow_length],
    theta=[arrow_direction],
    mode='markers',
    marker=dict(
        symbol="circle",  # Use a triangle marker to simulate an arrow#triangle-up
        size=12,
        color='black',
    ),
))

# Add arrow annotation
fig.add_annotation(
    x=arrow_x,
    y=arrow_y,
    ax=arrow_x,
    ay=arrow_y + 0.2,
    text="Arrow",
    showarrow=True,
    arrowhead=1,
    arrowcolor='black'
)

# Set polar plot layout
fig.update_layout(
    polar=dict(
        radialaxis=dict(
            visible=False,
            range=[0, 1]  # Set the range of the radial axis
        ),
        angularaxis=dict(
            visible=True,
            tickmode='array',
            tickvals=[0, 45, 90, 135, 180, 225, 270, 315],  # Set custom tick values for the angular axis
            ticktext=[ 'E', 'NE', 'N', 'NW', 'W', 'SW','S', 'SE'],  # Set custom tick labels for the angular axis
        )
    ),
    showlegend=False
)

fig.show()

199.09420064546472
0.4543195441356959
